In [1]:
from sqlalchemy import create_engine, Column, String, Integer, func, event, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm
from shapely.wkt import dumps

import orjson


In [2]:
# Create a base class for our declarative mapping
Base = declarative_base()

# Define your SQLAlchemy model
class GeometryModel(Base):
    __tablename__ = 'geometries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry('POLYGON'))

/tmp/ipykernel_87291/2237727833.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# Connect to the Spatialite database
db_path = 'sqlite:////tmp/blog/spatialite_ray.db'
engine = create_engine(db_path, echo=True)  # Set echo=True to see SQL commands being executed

In [4]:
# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')
    dbapi_connection.execute('SELECT InitSpatialMetaData(1);')

In [5]:
# Create the table
Base.metadata.create_all(engine)

# Start a session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2024-11-05 20:38:27,999 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-05 20:38:28,000 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("geometries")
2024-11-05 20:38:28,002 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-05 20:38:28,004 INFO sqlalchemy.engine.Engine COMMIT


InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


In [6]:
%%time
results = session.query(GeometryModel).all()

2024-11-05 20:38:28,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-05 20:38:28,026 INFO sqlalchemy.engine.Engine SELECT geometries.id AS geometries_id, geometries.name AS geometries_name, AsEWKB(geometries.geom) AS geometries_geom 
FROM geometries
2024-11-05 20:38:28,028 INFO sqlalchemy.engine.Engine [generated in 0.00188s] ()
CPU times: user 7min, sys: 18.4 s, total: 7min 19s
Wall time: 7min 31s


In [7]:
len(results)

1063260

In [8]:
%%time
# Detach all results from the session
# expunge_all() removes all objects from the session, making them detached. Detached objects are no longer tracked by the session, so changes made to them will not be automatically persisted to the database.
session.expunge_all()

CPU times: user 342 ms, sys: 10.1 ms, total: 352 ms
Wall time: 351 ms


In [9]:
%%time
## -- Convert from EWKB to Shapely
from shapely.geometry import shape
from shapely import wkb
for r in tqdm(results):
#    r.geom = wkb.loads(r.geom.data)
    r.geom = wkb.loads(str(r.geom))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1063260/1063260 [02:21<00:00, 7535.98it/s]

CPU times: user 2min 8s, sys: 13.2 s, total: 2min 21s
Wall time: 2min 21s


In [ ]:
%%time
from geoalchemy2.functions import ST_AsGeoJSON
results = session.query(ST_AsGeoJSON(GeometryModel.geom)).all()

In [ ]:
results[0]

In [ ]:
# %%time
# from geoalchemy2.functions import ST_AsEWKT
# results = session.query(ST_AsEWKT(GeometryModel.geom)).all()

In [ ]:
%%time
from geoalchemy2.functions import ST_Centroid
results = session.query(ST_Centroid(GeometryModel.geom)).all()

In [ ]:
len(results)

In [ ]:
results[0]

In [ ]:
%%time
from geoalchemy2.functions import ST_Centroid
results = session.query(ST_AsGeoJSON(ST_Centroid(GeometryModel.geom))).all()

In [ ]:
# %%time
results = session.query(GeometryModel.geom).limit(10).all()